In [30]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import model_selection
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
test=pd.read_csv("test.csv", nrows=5000)

In [2]:
train=pd.read_csv("train.csv", nrows=30000)

In [3]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()

In [25]:
train1.head()

,Country,Carrier,RefererUrl,publisherId,subPublisherId,advertiserCampaignId,OS_Android,OS_Desktop,OS_Mac OS X,OS_RIM OS,OS_Symbian OS,OS_Tizen,OS_Windows Phone OS,OS_iOS,TrafficType_A,TrafficType_M
0,56,2.0,551,481,57,6555.0,1,0,0,0,0,0,0,0,0,1
1,115,27.0,551,467,57,7510.0,1,0,0,0,0,0,0,0,1,0
2,56,4.0,551,259,57,12696.0,1,0,0,0,0,0,0,0,0,1
3,95,-1.0,551,202,57,735.0,1,0,0,0,0,0,0,0,1,0
4,84,-1.0,551,109,57,7802.0,1,0,0,0,0,0,0,0,0,0


In [ ]:
train1.OS.value_counts()

In [4]:
train1 = train.drop(['ConversionDate','ConversionPayOut','ConversionStatus','ID', 'Browser', 'Device', 'ClickDate','UserIp','Fraud'], axis=1)

In [5]:
train1 = pd.get_dummies(data=train1, columns=['OS','TrafficType'])
train1['RefererUrl'] = number.fit_transform(train['RefererUrl'].astype('str'))
train1['Country'] = number.fit_transform(train['Country'].astype('str'))
train1['subPublisherId'] = number.fit_transform(train['subPublisherId'].astype('str'))
train1['publisherId'] = number.fit_transform(train['publisherId'].astype('str'))
x = train1.values

In [6]:
x[[4567]]

array([[  8.90000000e+01,  -1.00000000e+00,   5.51000000e+02,
          4.39000000e+02,   5.70000000e+01,   3.31200000e+03,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00]])

In [26]:
train1.dtypes

Country                   int64
Carrier                 float64
RefererUrl                int64
publisherId               int64
subPublisherId            int64
advertiserCampaignId    float64
OS_Android                uint8
OS_Desktop                uint8
OS_Mac OS X               uint8
OS_RIM OS                 uint8
OS_Symbian OS             uint8
OS_Tizen                  uint8
OS_Windows Phone OS       uint8
OS_iOS                    uint8
TrafficType_A             uint8
TrafficType_M             uint8
dtype: object

In [6]:
y=train['ConversionPayOut'].ravel()

In [ ]:
t1 = test.drop('ID', axis=1)
t1 = pd.get_dummies(t1)
testA = t1.values

In [31]:
xgb = xgb.XGBRegressor(max_depth=2, learning_rate=0.01, n_estimators=500, silent=True, objective='reg:linear',
                      min_child_weight=1, subsample=0.5, colsample_bytree=0.5, gamma = 4, 
                      scale_pos_weight=0.5, 
                      seed=1000, missing=None, )

In [52]:
rf = RandomForestRegressor(n_estimators=5)
svr = SVR(kernel='linear')
lasso = Lasso(alpha=2.0, fit_intercept=True, normalize=True,max_iter=5000)
mlp = MLPRegressor(hidden_layer_sizes=(17,15,10), activation='relu',alpha=0.0001,  
                   max_iter=500, shuffle=True, random_state=100)
gb = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05)
stack = StackingCVRegressor(regressors=(gb,lasso, rf, mlp),
                            meta_regressor=xgb)

In [ ]:
st = model_selection.cross_val_score(svr, x, y, 
                                              cv=5,scoring='neg_mean_squared_error',n_jobs = -1) #Stacking model CV calculation
st.mean()

In [42]:
sta = model_selection.cross_val_score(stack, x, y, 
                                              cv=2,scoring='neg_mean_squared_error',n_jobs = -1) #Stacking model CV calculation
sta.mean()

-285.60096890596674